In [17]:
nums = [2,7,11,15]
target = 9

In [15]:
def two_sum(nums, target):
    num_map={}

    for i, num in enumerate(nums):
        complement = target - num #complement that adds up to the target

        if complement in num_map: #if the complement exists in thmap, return the indices
            return [num_map[complement], i]

        #Otherwise, store the current number and its index in the map
        num_map[num] = i
    return []

In [18]:
two_sum(nums,target)

[0, 1]

In [19]:
!pip install pandas openpyxl pymupdf

Looking in indexes: https://artifacts.benteler.net/repository/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 14.7 MB/s eta 0:00:0000:0100:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [26]:
!pip install pymupdf

Looking in indexes: https://artifacts.benteler.net/repository/pypi/simple


In [22]:
import os

# Path to the Excel file
excel_path = "5. May 2025 Surcharges COPY.xlsx"

# Folder where your downloaded PDFs are saved
pdf_folder = "5. May PDFs"  # Make sure this folder exists in your working directory

# Output Excel file name
output_excel_path = "Surcharges_with_PDF_Text.xlsx"


In [24]:
import pandas as pd

# Load the Excel file
df = pd.read_excel(excel_path, sheet_name="Links")  # Change sheet name if needed

# Preview the first few rows
df.head()


,ONLINE SITES,Website,Notes,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,ACS,https://acsindustries.com/surcharges/archive/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Kallas Edelstahl,Alloy surcharges | Kallas stainless steel GmbH...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Rohr Mertl,Valid alloy surcharges – Rohr Mertl,Condition YKG2 - use tube on SC sheet,NaN,NaN,NaN,NaN,NaN,NaN
3,Carpenter,Raw Materials Surcharge | Carpenter Technology,click on Raw Materials Surchasrges - Cast/Wrought,NaN,NaN,NaN,NaN,NaN,NaN
4,Cleveland Cliffs (AK Steel),For Customers :: Cleveland-Cliffs Inc. (CLF) (...,"Scroll down to ""stainless surcharges"" --> Hit ...",NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
import fitz  # PyMuPDF
import os

pdf_folder = "5. May PDFs"
pdf_texts = {}

for filename in os.listdir(pdf_folder):
    if filename.lower().endswith(".pdf"):
        filepath = os.path.join(pdf_folder, filename)
        with fitz.open(filepath) as doc:
            text = "\n".join(page.get_text() for page in doc)
            pdf_texts[filename] = text.lower()  # Store text in lowercase for easier matching

# Preview first few PDF names and text lengths
list(pdf_texts.items())[:3]


[('Outokumpo_Precision_May.pdf',
  'monthly alloy surcharges for precision strip\nmonthly alloy surcharges may 2025\nsteel designations\ncurrency / tonne\nupdated:2025-04-22\noutokumpu\nen\nastm\neur\ngbp\nusd\nsek\nnok\ndkk\nczk\npln\nhuf\ncnh\nzar\nmoda\nmoda 430/4016\n1.4016\n430\n1221\n1034\n1343\n13364\n14222\n9116\n30576\n5165\n493382\n9778\n25132\nmoda 439/4510\n1.4510\n439\n1210\n1024\n1330\n13234\n14084\n9027\n30279\n5115\n488591\n9683\n24888\nmoda 409/4512\n1.4512\n409\n1045\n885\n1150\n11437\n12172\n7801\n26167\n4420\n422239\n8368\n21508\ncore\ncore 434/4113\n1.4113\n434\n1721\n1457\n1893\n18831\n20041\n12845\n43085\n7278\n695222\n13779\n35413\ncore 304/4301\n1.4301\n304\n2382\n2015\n2619\n26056\n27730\n17773\n59615\n10070\n961960\n19065\n49001\ncore 305/4303\n1.4303\n305\n3005\n2543\n3305\n32878\n34991\n22427\n75226\n12707\n1213849\n24057\n61831\ncore 304l/4306\n1.4306\n304l\n2603\n2203\n2863\n28478\n30307\n19425\n65157\n11006\n1051376\n20837\n53555\ncore 304l/4307\n1.4307\

In [11]:
import pandas as pd

excel_path = "5. May 2025 Surcharges COPY.xlsx"
df = pd.read_excel(excel_path, sheet_name="May 2025")

In [12]:
print(df.columns.tolist())

['Cond Type', 'Description of Calucation', 'UoM', 'Rates', 'Rates to be posted', 'Date Range', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8']


In [13]:
desc_col = "Description of Calucation"  # Check exact spelling in Excel
df["Matched PDF Content"] = ""

for idx, row in df.iterrows():
    description = str(row[desc_col]).lower()
    
    # Extract likely keywords (e.g., "304L", "309SS")
    keywords = [word for word in description.split() if any(char.isdigit() for char in word)]
    
    matched = False
    for pdf_name, text in pdf_texts.items():
        if any(kw in text for kw in keywords):
            df.at[idx, "Matched PDF Content"] = text[:3000]  # Limit to 3000 characters
            matched = True
            break

    if not matched:
        df.at[idx, "Matched PDF Content"] = "No matching PDF found"

df.head()


,Cond Type,Description of Calucation,UoM,Rates,Rates to be posted,Date Range,Unnamed: 6,Unnamed: 7,Unnamed: 8,Matched PDF Content
0,YSW2,A286 Aerospace HTA - GW x MSR Quarterly,LBS,3.5500,3550.0,4/01/2025 - 7/31/2025,Sent by MacLean Fogg,NaN,NaN,"acs industries,inc.\nraw material surcharge po..."
1,YMD2,ACS 304L - GW x MSR including 6% scrap,LBS,1.0158,1015.8,4/01/2025 - 4/31/2025,NaN,NaN,NaN,monthly alloy surcharges for precision strip\n...
2,YMC2,ACS 309SS - GW x MSR including 6% scrap,LBS,1.2669,1266.9,4/01/2025 - 4/31/2025,NaN,NaN,NaN,"acs industries,inc.\nraw material surcharge po..."
3,YMG2,ACS 310SS - GW X MSR,LBS,1.7068,1706.8,4/01/2025 - 4/31/2025,NaN,NaN,NaN,No matching PDF found
4,YME2,ACS 316Ti - GW X MSR including 6% scrap,LBS,1.6501,1650.1,4/01/2025 - 4/31/2025,Using 316 Ti,NaN,NaN,monthly alloy surcharges for precision strip\n...


In [15]:
output_excel_path = "Updated_May_2025_Surcharges.xlsx"
df.to_excel(output_excel_path, index=False)
print(f"Saved to: {output_excel_path}")


Saved to: Updated_May_2025_Surcharges.xlsx


In [16]:
df.to_excel(output_excel_path, index=False)
print(f"Saved to: {output_excel_path}")


Saved to: Updated_May_2025_Surcharges.xlsx


# Monthly Surcharge PDF Matching

This notebook extracts and matches PDF content against descriptions from a monthly surcharge Excel sheet.

## How to Use

1. Upload the monthly Excel file (e.g., `June_2025_Surcharges.xlsx`).
2. Upload PDFs into a folder (e.g., `June_PDFs`).
3. Update variables in the notebook:
   - `pdf_folder`(make a folder and put the PDFs in it)
   - `excel_path` (make sure the excel is also in jupyter)
   - `output_excel_path`(change to June for example for next month)
   - `sheet_name` (Tab on the excel e.g., June 2025
   - `desc_col` (stays the same)
4. Run the notebook.
5. Check `Updated_<month>_Surcharges.xlsx` for output.

## Dependencies

- pandas
- PyMuPDF (`pip install pymupdf`)
